movie 데이터셋에 장르 추가하기

In [1]:
import pandas as pd
import json

In [2]:
u_cols =  ['movie id', 'movie title', 'release date', 'video release date',
'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
'Thriller', 'War', 'Western'
]
new = pd.read_csv('../data/origin_new_data/u.item', sep='|', names=u_cols, encoding='latin-1').drop('unknown', axis='columns')
new.head(2)

,movie id,movie title,release date,video release date,IMDb URL,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
genre_dict = {}
for _, row in new.iterrows():
    title = row['movie title']
    genres = [genre.lower() for genre in new.columns[5:] if row[genre]==1]
    genre_dict[title]=genres

In [4]:
genre_df = pd.DataFrame([
    {'title': title, 'genre': ', '.join(genres)} for title, genres in genre_dict.items()
])

# CSV로 저장
genre_df.to_csv('../data/genre_data.csv', index=False, encoding='utf-8')

print("CSV 파일로 저장 완료!")

CSV 파일로 저장 완료!


In [193]:
for key, value in genre_dict.items():
    for v in value:
        v.strip('\'')
    
genre_dict

{'Toy Story (1995)': ['animation', "children's", 'comedy'],
 'GoldenEye (1995)': ['action', 'adventure', 'thriller'],
 'Four Rooms (1995)': ['thriller'],
 'Get Shorty (1995)': ['action', 'comedy', 'drama'],
 'Copycat (1995)': ['crime', 'drama', 'thriller'],
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': ['drama'],
 'Twelve Monkeys (1995)': ['drama', 'sci-fi'],
 'Babe (1995)': ["children's", 'comedy', 'drama'],
 'Dead Man Walking (1995)': ['drama'],
 'Richard III (1995)': ['drama', 'war'],
 'Seven (Se7en) (1995)': ['crime', 'thriller'],
 'Usual Suspects, The (1995)': ['crime', 'thriller'],
 'Mighty Aphrodite (1995)': ['comedy'],
 'Postino, Il (1994)': ['drama', 'romance'],
 "Mr. Holland's Opus (1995)": ['drama'],
 'French Twist (Gazon maudit) (1995)': ['comedy', 'romance'],
 'From Dusk Till Dawn (1996)': ['action',
  'comedy',
  'crime',
  'horror',
  'thriller'],
 'White Balloon, The (1995)': ['drama'],
 "Antonia's Line (1995)": ['drama'],
 'Angels and Insects (1995)': ['dram

In [205]:
with open('../data/movie/valid.json', 'r') as file:
    data = json.load(file)

In [206]:
for entry in data:
    print(entry)

{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': 'User Preference: "Silence of the Lambs, The (1991)", "Indiana Jones and the Last Crusade (1989)", "Taxi Driver (1976)", "My Favorite Year (1982)", "Titanic (1997)"\nUser Unpreference: "Absolute Power (1997)", "Escape from L.A. (1996)", "Batman & Robin (1997)", "English Patient, The (1996)", "Tomorrow Never Dies (1997)"\nWhether the user will like the target movie "Scream (1996)"?', 'output': 'No.'}
{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': 'User Preference: "Silence of the Lambs, The (1991)", "Indiana Jones and the Last Crusade (1989)", "Taxi Driver (1976)", "My Favorite Year (1982)", "Titanic (1997)"\nUser Unpreference: "Escape from L.A. (1996)", "Batman & Robin (1997)", "English Patient, The (1996)", "Tomorrow Nev

In [207]:
import re

def add_genres_to_input_list(input_text):
    preference_pattern = r'User Preference:\s*"(.*?)"\n'
    preferences = re.findall(preference_pattern, input_text)
    preferences_list = preferences[0].split('", "') if preferences else []
    
    unpreference_pattern = r'User Unpreference:\s*"(.*?)"\n'
    unpreferences = re.findall(unpreference_pattern, input_text)
    unpreferences_list = unpreferences[0].split('", "') if unpreferences else []
    
    target_pattern = r'Whether the user will like the target movie\s*"(.*?)"'
    target_movie = re.search(target_pattern, input_text).group(1)
    
    def format_mv(movie):
        genres = genre_dict.get(movie.strip(), [])
        genre_str = ', '.join(genres) if genres else 'None'
        return f'{movie}:[{genre_str}]'
    
    like_mv = [format_mv(movie) for movie in preferences_list]
    unlike_mv = [format_mv(movie) for movie in unpreferences_list]
    #target_mv = [f'{target_movie} : {genre_dict.get(target_movie)}']

    update_input = (
        f'User Preference: {", ".join(like_mv)}\n' if like_mv else ''
    )
    update_input += (
        f'User Unpreference: {", ".join(unlike_mv)}\n' if unlike_mv else ''
    )
    update_input += (
        f'Whether the user will like the target movie {target_movie} : {genre_dict.get(target_movie)}?'
    )
    
    return update_input

for entry in data:
    entry['input'] = add_genres_to_input_list(entry['input'])
    print(entry)

{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': "User Preference: Silence of the Lambs, The (1991):[drama, thriller], Indiana Jones and the Last Crusade (1989):[action, adventure], Taxi Driver (1976):[drama, thriller], My Favorite Year (1982):[comedy], Titanic (1997):[action, drama, romance]\nUser Unpreference: Absolute Power (1997):[mystery, thriller], Escape from L.A. (1996):[action, adventure, sci-fi, thriller], Batman & Robin (1997):[action, adventure, crime], English Patient, The (1996):[drama, romance, war], Tomorrow Never Dies (1997):[action, romance, thriller]\nWhether the user will like the target movie Scream (1996) : ['horror', 'thriller']?", 'output': 'No.'}
{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': "User Preference: Silence of the Lambs, The (1991):[d

In [208]:
for entry in data:
    entry['instruction'] = 'Given the user\'s preference and unpreference, along with the genres of each movie, identify whether the user will like the target movie by answering "Yes." or "No.".'

In [209]:
with open('../data/movie_genre_list_instruction/valid.json', 'w') as file:
    json.dump(data, file, indent=4)

print("new json create!")

new json create!


In [58]:
import re

def add_genres_to_input(input_text):
    preference_pattern = r'User Preference:\s*"(.*?)"\n'
    preferences = re.findall(preference_pattern, input_text)
    preferences_list = preferences[0].split('", "') if preferences else []
    
    unpreference_pattern = r'User Unpreference:\s*"(.*?)"\n'
    unpreferences = re.findall(unpreference_pattern, input_text)
    unpreferences_list = unpreferences[0].split('", "') if unpreferences else []
    
    target_pattern = r'Whether the user will like the target movie\s*"(.*?)"'
    target_movie = re.search(target_pattern, input_text).group(1)
    
    like_mv = [f'{movie} the genre of {", ".join(genre_dict.get(movie,[]))}' for movie in preferences_list]
    unlike_mv = [f'{movie} the genre of {", ".join(genre_dict.get(movie,[]))}' for movie in unpreferences_list]
    target_mv = f'{target_movie} the genre of {", ".join(genre_dict.get(target_movie,[]))}' if target_movie else ""

    update_input = (
        f'User Preference: {" / ".join(like_mv)}\n' if like_mv else ''
    )
    update_input += (
        f'User Unpreference: {" / ".join(unlike_mv)}\n' if unlike_mv else ''
    )
    update_input += (
        f'Whether the user will like the target movie {target_mv}?' if target_mv else ''
    )
    
    return update_input

In [ ]:
for entry in data:
    entry['input'] = add_genres_to_input(entry['input'])

In [60]:
for entry in data:
    print(entry)

{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': 'User Preference: Paris, Texas (1984) the genre of drama / Rebel Without a Cause (1955) the genre of drama / Return of the Pink Panther, The (1974) the genre of comedy / Ace Ventura: Pet Detective (1994) the genre of comedy / Magnificent Seven, The (1954) the genre of action, drama, western / Star Trek: The Wrath of Khan (1982) the genre of action, adventure, sci-fi / Cat People (1982) the genre of horror / Orlando (1993) the genre of drama / Dave (1993) the genre of comedy, romance\nUser Unpreference: Kalifornia (1993) the genre of drama, thriller\nWhether the user will like the target movie Perez Family, The (1995) the genre of comedy, romance?', 'output': 'No.'}
{'instruction': 'Given the user\'s preference and unpreference, identify whether the user will like the target movie by answering "Yes." or "No.".', 'input': 'User Pref

In [118]:
with open('../data/movie_genre/test.json', 'w') as file:
    json.dump(data, file, indent=4)

print("new json create!")

new json create!


instruction 변경

In [88]:
with open('../data/new/valid.json', 'r') as new_file:
    new_data = json.load(new_file)

In [89]:
for entry in new_data:
    entry['instruction'] = 'Given the user\'s preference and unpreference, along with the genres of each movie, identify whether the user will like the target movie by answering "Yes." or "No.".'

In [90]:
with open('../data/new_genre_instruction/valid.json', 'w') as file:
    json.dump(new_data, file, indent=4)

print("new json create!")

new json create!
